### Colab Link
https://colab.research.google.com/drive/1U1kOIaO_tNa0AgDPj1-62uAebJKu_vnD?usp=sharing

In [1]:
!gdown 1sl8n7-Qdv3UHcHABCJ60bVkqAVzzjjry

Downloading...
From: https://drive.google.com/uc?id=1sl8n7-Qdv3UHcHABCJ60bVkqAVzzjjry
To: /content/books.csv
100% 1.56M/1.56M [00:00<00:00, 42.2MB/s]


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


import os

In [3]:
data = pd.read_csv("books.csv", error_bad_lines = False)
data.head()

<ipython-input-3-bc00f3337d22>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv("books.csv", error_bad_lines = False)
Skipping line 3350: expected 12 fields, saw 13
Skipping line 4704: expected 12 fields, saw 13
Skipping line 5879: expected 12 fields, saw 13
Skipping line 8981: expected 12 fields, saw 13



,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic


In [5]:
new_data = data.copy()

In [6]:
def fun_only_author(text):
    arlen = text.split('/')
    return arlen[0]

In [7]:
new_data['only_author'] = new_data['authors'].apply(lambda x : fun_only_author(x))

In [8]:
total_rating = new_data.drop_duplicates(subset=['only_author', 'title'], keep='first')
total_rating = total_rating.groupby(by=['only_author']).agg({'average_rating': ['sum']})
total_rating.columns = ['total_rating']
total_rating.reset_index(inplace=True)
total_rating = total_rating.sort_values(by=['total_rating'], ascending=False)
total_rating

,only_author,total_rating
3753,Stephen King,250.79
3006,P.G. Wodehouse,187.31
4161,William Shakespeare,178.45
1603,J.R.R. Tolkien,172.22
3536,Rumiko Takahashi,163.28
...,...,...
146,Andrew Hunt,0.00
1654,James Craig Holte,0.00
1751,Jasmine C.M. Luk,0.00
935,Dobrica Erić,0.00


In [9]:
total_book = new_data.groupby(by=['only_author']).agg({'title': ['nunique']})
total_book.columns = ['total_book']
total_book.reset_index(inplace=True)
total_book = total_book.sort_values(by=['total_book'], ascending=False)
total_book

,only_author,total_book
3753,Stephen King,63
4161,William Shakespeare,45
3006,P.G. Wodehouse,45
29,Agatha Christie,41
1603,J.R.R. Tolkien,41
...,...,...
1727,Jane Parker Resnick,1
1729,Jane Robins,1
1731,Jane Walmsley,1
1732,Jane Werner Watson,1


In [10]:
avg_author = pd.merge(total_book, total_rating, on='only_author', how='outer')
avg_author['average_rating'] = round(avg_author['total_rating'] / avg_author['total_book'], 2)
avg_author = avg_author[avg_author['total_book'] > 26]
avg_author = avg_author.sort_values(by=['average_rating'], ascending=False)
avg_author

,only_author,total_book,total_rating,average_rating
4,J.R.R. Tolkien,41,172.22,4.20
7,Rumiko Takahashi,39,163.28,4.19
11,C.S. Lewis,32,134.02,4.19
2,P.G. Wodehouse,45,187.31,4.16
12,Terry Pratchett,31,127.55,4.11
9,Roald Dahl,35,141.66,4.05
6,Mercedes Lackey,40,160.11,4.00
18,Mark Twain,27,107.68,3.99
0,Stephen King,63,250.79,3.98
1,William Shakespeare,45,178.45,3.97


## Weighted Rating calculation¶

In [11]:
total_vote = new_data.drop_duplicates(subset=['only_author', 'title'], keep='first')
total_vote.reset_index(inplace=True)
total_vote = total_vote[['only_author', 'title', 'average_rating', 'ratings_count']]
total_vote

,only_author,title,average_rating,ratings_count
0,J.K. Rowling,Harry Potter and the Half-Blood Prince (Harry ...,4.57,2095690
1,J.K. Rowling,Harry Potter and the Order of the Phoenix (Har...,4.49,2153167
2,J.K. Rowling,Harry Potter and the Chamber of Secrets (Harry...,4.42,6333
3,J.K. Rowling,Harry Potter and the Prisoner of Azkaban (Harr...,4.56,2339585
4,J.K. Rowling,Harry Potter Boxed Set Books 1-5 (Harry Potte...,4.78,41428
...,...,...,...,...
10416,William T. Vollmann,Expelled from Eden: A William T. Vollmann Reader,4.06,156
10417,William T. Vollmann,You Bright and Risen Angels,4.08,783
10418,William T. Vollmann,The Ice-Shirt (Seven Dreams #1),3.96,820
10419,William T. Vollmann,Poor People,3.72,769


In [12]:
C = total_vote.average_rating.mean()
C

3.933244410325305

In [13]:
m = total_vote.ratings_count.quantile(0.9)
m

24967.0

In [14]:
total_vote = total_vote[total_vote['ratings_count'] >= m]
total_vote.head()

,only_author,title,average_rating,ratings_count
0,J.K. Rowling,Harry Potter and the Half-Blood Prince (Harry ...,4.57,2095690
1,J.K. Rowling,Harry Potter and the Order of the Phoenix (Har...,4.49,2153167
3,J.K. Rowling,Harry Potter and the Prisoner of Azkaban (Harr...,4.56,2339585
4,J.K. Rowling,Harry Potter Boxed Set Books 1-5 (Harry Potte...,4.78,41428
6,J.K. Rowling,Harry Potter Collection (Harry Potter #1-6),4.73,28242


In [15]:
def weighted_rating(x, m=m, C=C):
    v = x['ratings_count']
    R = x['average_rating']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [16]:
total_vote['score'] = total_vote.apply(weighted_rating, axis=1)

<ipython-input-16-623fbe960cbc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_vote['score'] = total_vote.apply(weighted_rating, axis=1)


In [17]:
total_vote = total_vote.sort_values(by='score', ascending=False).head(20)
total_vote

,only_author,title,average_rating,ratings_count,score
0,J.K. Rowling,Harry Potter and the Half-Blood Prince (Harry ...,4.57,2095690,4.562503
3,J.K. Rowling,Harry Potter and the Prisoner of Azkaban (Harr...,4.56,2339585,4.553382
1,J.K. Rowling,Harry Potter and the Order of the Phoenix (Har...,4.49,2153167,4.483618
4,J.K. Rowling,Harry Potter Boxed Set Books 1-5 (Harry Potte...,4.78,41428,4.461588
20,J.R.R. Tolkien,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,4.59,101233,4.460070
3955,Art Spiegelman,The Complete Maus,4.55,111475,4.437142
6137,Bill Watterson,The Complete Calvin and Hobbes,4.82,32213,4.432808
3965,J.R.R. Tolkien,The Two Towers (The Lord of the Rings #2),4.44,593467,4.419542
269,Hiromu Arakawa,Fullmetal Alchemist Vol. 1 (Fullmetal Alchemi...,4.50,111091,4.395999
6138,Bill Watterson,The Calvin and Hobbes Tenth Anniversary Book,4.63,49122,4.395203


# Recommendation Engine
## Creating Features

In [18]:
new_data.loc[ (new_data['average_rating'] >= 0) & (new_data['average_rating'] <= 1), 'rating_between'] = "between_0_to_1"
new_data.loc[ (new_data['average_rating'] > 1) & (new_data['average_rating'] <= 2), 'rating_between'] = "between_1_to_2"
new_data.loc[ (new_data['average_rating'] > 2) & (new_data['average_rating'] <= 3), 'rating_between'] = "between_2_to_3"
new_data.loc[ (new_data['average_rating'] > 3) & (new_data['average_rating'] <= 4), 'rating_between'] = "between_3_to_4"
new_data.loc[ (new_data['average_rating'] > 4) & (new_data['average_rating'] <= 5), 'rating_between'] = "between_4_to_5"

In [19]:
new_data.head(2)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,only_author,rating_between
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,J.K. Rowling,between_4_to_5
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,J.K. Rowling,between_4_to_5


In [20]:
trial = new_data[['average_rating', 'ratings_count']]
data_model = np.asarray([np.asarray(trial['average_rating']), np.asarray(trial['ratings_count'])]).T
data_model

array([[4.570000e+00, 2.095690e+06],
       [4.490000e+00, 2.153167e+06],
       [4.420000e+00, 6.333000e+03],
       ...,
       [3.960000e+00, 8.200000e+02],
       [3.720000e+00, 7.690000e+02],
       [3.910000e+00, 1.130000e+02]])

In [21]:
from sklearn.cluster import KMeans

In [22]:
# Elbow method
score = []
x = data_model
for cluster in range(1,41):
    kmeans = KMeans(n_clusters = cluster, init="k-means++", random_state=40)
    kmeans.fit(x)
    score.append(kmeans.inertia_)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

In [28]:
rating_between_df = new_data['rating_between'].str.get_dummies(sep=",")
rating_between_df.head()

,between_0_to_1,between_1_to_2,between_2_to_3,between_3_to_4,between_4_to_5
0,0,0,0,0,1
1,0,0,0,0,1
2,0,0,0,0,1
3,0,0,0,0,1
4,0,0,0,0,1


In [29]:
lang_df = new_data['language_code'].str.get_dummies(sep=",")
lang_df.head()

,ale,ara,en-CA,en-GB,en-US,eng,enm,fre,ger,gla,...,nl,nor,por,rus,spa,srp,swe,tur,wel,zho
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
engine_features = pd.concat([rating_between_df, lang_df, new_data['average_rating'], new_data['ratings_count']], axis=1)
engine_features.head()

,between_0_to_1,between_1_to_2,between_2_to_3,between_3_to_4,between_4_to_5,ale,ara,en-CA,en-GB,en-US,...,por,rus,spa,srp,swe,tur,wel,zho,average_rating,ratings_count
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4.57,2095690
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4.49,2153167
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4.42,6333
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4.56,2339585
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4.78,41428


In [31]:
from sklearn.preprocessing import MinMaxScaler

In [32]:
min_max_scaler = MinMaxScaler()
engine_features = min_max_scaler.fit_transform(engine_features)

In [33]:
from sklearn import neighbors

In [34]:
engine_model = neighbors.NearestNeighbors(n_neighbors=10, algorithm='ball_tree')

In [35]:
engine_model.fit(engine_features)

NearestNeighbors(algorithm='ball_tree', n_neighbors=10)

In [36]:
dist, idlist = engine_model.kneighbors(engine_features)

In [37]:
def book_recommendation_engine(book_name):
    book_list_name = []
    book_id = new_data[new_data['title'] == book_name].index
    book_id = book_id[0]
    for newid in idlist[book_id]:
        book_list_name.append(new_data.loc[newid].title)
    return book_list_name

In [38]:
book_list_name = book_recommendation_engine('The Hobbit')
book_list_name

['The Hobbit',
 'The Voyage of the Jerle Shannara Trilogy (Voyage of the Jerle Shannara  #1-3)',
 'Tsubasa: RESERVoir CHRoNiCLE  Vol. 8',
 'The Complete Plays',
 'Ariel: The Restored Edition',
 'The Forbidden (Vampire Huntress  #5)',
 'Agatha Christie: An Autobiography',
 'What Work Is',
 'How Europe Underdeveloped Africa',
 'Exclusion & Embrace: A Theological Exploration of Identity  Otherness  and Reconciliation']

# Saving the data

In [39]:
import json

# Convert NumPy arrays to Python lists
idlist_as_list = idlist.tolist()
data_as_list = new_data.to_dict(orient='records')

# Save idlist and data as JSON
with open('idlist.json', 'w') as idlist_json_file:
    json.dump(idlist_as_list, idlist_json_file)

with open('data.json', 'w') as data_json_file:
    json.dump(data_as_list, data_json_file)